In [10]:
import requests
import pandas as pd

def decode_sex(sex_code):
    return {"1": "MALE", "2": "FEMALE"}.get(str(sex_code), "UNKNOWN")

drug_list = [
    "ibuprofen",
    "acetaminophen",
    "lisinopril"
    # add more as needed
]

records = []

for drug in drug_list:
    print(f"Fetching data for: {drug}")
    for skip in range(0, 1000, 100):
        params = {
            "search": f'patient.drug.medicinalproduct:"{drug}"',
            "limit": 100,
            "skip": skip,
            "sort": "receivedate:desc"
        }

        try:
            response = requests.get("https://api.fda.gov/drug/event.json", params=params)
            response.raise_for_status()
            results = response.json().get("results", [])
            if not results:
                print(f"No more data for {drug} after {skip} records.")
                break
        except Exception as e:
            print(f"Error fetching data for {drug} at skip={skip}: {e}")
            break

        for entry in results:
            patient = entry.get("patient", {})
            record = {
                "query_drug": drug.upper(),
                "report_id": entry.get("safetyreportid"),
                "received_date": entry.get("receivedate"),
                "serious": entry.get("serious"),
                "age": patient.get("patientonsetage"),
                "age_unit": patient.get("patientonsetageunit"),
                
                "sex": decode_sex(patient.get("patientsex")),
                "reactions": " | ".join([
                    r.get("reactionmeddrapt", "").upper()
                    for r in patient.get("reaction", [])
                ]),
                "outcome_death": entry.get("seriousnessdeath", 0)
            }
            records.append(record)

# Create DataFrame
df = pd.DataFrame(records)

# Convert received_date to datetime
df["received_date"] = pd.to_datetime(df["received_date"], format="%Y%m%d", errors="coerce")

# Sort by received_date descending
df_sorted = df.sort_values(by="received_date", ascending=False)


#changing the name to adverse_sorted

adverse_effects = df.sort_values(by="received_date", ascending=False)

#STEP 2: Mapping to Differnt Countries - building a mini look table to find ATC codes 
#These have to be manually inputed b/c no downloadable database exists
# Load your CSV
atc_lookup = pd.read_csv('/Users/dustinh/globalhealth/pipeline/mapping_layer_atc.csv')

# Make sure columns are uppercase for consistency
adverse_effects['query_drug'] = adverse_effects['query_drug'].str.upper()
atc_lookup['drug_name'] = atc_lookup['drug_name'].str.upper()
atc_lookup['atc_code'] = atc_lookup['atc_code'].str.upper()

# Create a mapping Series from atc_lookup DataFrame
mapping_series = atc_lookup.set_index('drug_name')['atc_code']

# Use map with this Series
adverse_effects['atc_code'] = adverse_effects['query_drug'].map(mapping_series)


#STEP # : now let's add our mapping layer 

country_approval = pd.read_csv('/Users/dustinh/globalhealth/pipeline/mapping_layer_country_approval_new_idea.csv')

country_approval
# Make sure the ATC code columns are uppercase for consistency
country_approval['atc_code'] = country_approval['atc_code'].str.upper()
adverse_effects['atc_code'] = adverse_effects['atc_code'].str.upper()

# Merge on 'atc_code' — this will add the country approval info to your adverse_effects
merged_df = adverse_effects.merge(country_approval, on='atc_code', how='left')

# Check the merged (final) result 
merged_df


Fetching data for: ibuprofen
Fetching data for: acetaminophen
Fetching data for: lisinopril


,query_drug,report_id,received_date,serious,age,age_unit,sex,reactions,outcome_death,atc_code,generic_name,country,approved,notes_and_source
0,IBUPROFEN,25138898,2025-03-31,1,None,None,FEMALE,SPINAL CORD INFECTION | PERICARDITIS,2,M01AE01,Ibuprofen,USA (FDA),Y,Listed in Drugs@FDA
1,IBUPROFEN,25138898,2025-03-31,1,None,None,FEMALE,SPINAL CORD INFECTION | PERICARDITIS,2,M01AE01,Ibuprofen,EU (EMA),Y,Available in EMA database
2,IBUPROFEN,25138898,2025-03-31,1,None,None,FEMALE,SPINAL CORD INFECTION | PERICARDITIS,2,M01AE01,Ibuprofen,Canada,Y,Health Canada Drug Product Database
3,IBUPROFEN,25138898,2025-03-31,1,None,None,FEMALE,SPINAL CORD INFECTION | PERICARDITIS,2,M01AE01,Ibuprofen,India,Y,CDSCO list
4,ACETAMINOPHEN,25142565,2025-03-31,1,None,None,FEMALE,HALLUCINATION | HIP FRACTURE | PARAESTHESIA | ...,2,N02BE01,Acetaminophen,USA (FDA),Y,Listed in Drugs@FDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,IBUPROFEN,25026219,2025-03-01,1,None,None,FEMALE,BREAST CANCER | BREAST CYST | ERYTHEMA | PRURI...,2,M01AE01,Ibuprofen,India,Y,CDSCO list
11996,LISINOPRIL,25025866,2025-03-01,1,49,801,FEMALE,DEHYDRATION | DECREASED APPETITE | IMPAIRED GA...,2,C09AA03,Lisinopril,USA (FDA),Y,Approved since 1987 (Drugs@FDA)
11997,LISINOPRIL,25025866,2025-03-01,1,49,801,FEMALE,DEHYDRATION | DECREASED APPETITE | IMPAIRED GA...,2,C09AA03,Lisinopril,EU (EMA),Y,Approved under several brand names
11998,LISINOPRIL,25025866,2025-03-01,1,49,801,FEMALE,DEHYDRATION | DECREASED APPETITE | IMPAIRED GA...,2,C09AA03,Lisinopril,Canada,Y,Health Canada listing
